In [1]:
import pymc3 as pm
import arviz as az
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import copy
import warnings
from theano import shared,tensor
import time 
from datetime import datetime
from sklearn.metrics import mean_squared_error
import pickle

In [12]:
df=pd.read_csv('/Users/pryang/Documents/貝氏資料分析介紹/Project/Bayesian-Analysis/EPA_OD_202304.csv')
truth=copy.deepcopy(df)
df=df[df['PublishTime']>='2023-04-24 00:00:00']
df.head()

,SiteName,County,AQI,Pollutant,Status,SO2,CO,CO_8hr,O3,O3_8hr,...,NO2,NOx,NO,WindSpeed,WindDirec,PublishTime,SO2_AVG,Longitude,Latitude,SiteId
42427,永和,新北市,89.0,懸浮微粒,普通,0.0,0.30,0.3,40.7,43.7,...,10.5,10.8,0.2,1.1,95,2023-04-24 00:00:00,0.0,121.516306,25.017000,70
42428,復興,高雄市,77.0,細懸浮微粒,普通,1.6,0.38,0.3,24.7,43.7,...,18.4,18.5,0.0,0.7,284,2023-04-24 00:00:00,1.0,120.312017,22.608711,71
42429,富貴角,新北市,100.0,懸浮微粒,普通,0.2,0.14,0.1,61.5,64.1,...,1.2,1.2,0.0,4.8,99,2023-04-24 00:00:00,0.0,121.536569,25.296817,84
42430,花蓮,花蓮縣,67.0,細懸浮微粒,普通,0.7,0.27,0.2,44.7,49.2,...,4.7,4.5,0.0,0.3,311,2023-04-24 00:00:00,0.0,121.599769,23.971306,63
42431,宜蘭,宜蘭縣,77.0,懸浮微粒,普通,1.0,0.20,0.2,42.8,50.7,...,2.9,3.4,0.5,0.3,221,2023-04-24 00:00:00,0.0,121.746394,24.747917,65


In [13]:
features=['AQI','PM10','PM2.5_AVG','PM2.5','SiteId']
for feature in features:
    print(f"{feature} has nan value: ",df[feature].isna().any())
    # print(df[df[feature].isna()].index)

AQI has nan value:  True
PM10 has nan value:  True
PM2.5_AVG has nan value:  True
PM2.5 has nan value:  True
SiteId has nan value:  False


In [14]:
def preprocessing(data):
    data=data[features]
    data.dropna(inplace=True)
    return data
df=preprocessing(df)

<ipython-input-14-7e5edb5d69f7>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.dropna(inplace=True)


In [5]:
Site=list(df.SiteId.unique())
Site.sort()

In [42]:
SiteTrace={}
number=1
for idx,s in enumerate(Site):
    with pm.Model() as model:
        X0=pm.Data('X0',df[df.SiteId==s][features[0]])
        X1=pm.Data('X1',df[df.SiteId==s][features[1]])
        X2=pm.Data('X2',df[df.SiteId==s][features[2]])
        y=pm.Data('y',df[df.SiteId==s][features[3]])
        alpha=pm.Normal('alpha',mu=0,sd=10)
        beta=pm.Normal('beta',mu=0,sd=10,shape=3)
        esp=pm.HalfNormal('esp',sd=10)
        mu=pm.Deterministic('mu',alpha+beta[0]*X0+beta[1]*X1+beta[2]*X2)
        y_pred=pm.Normal('y_pred',mu=mu,sd=esp,observed=y)
        trace=pm.sample()
        ppc=pm.sample_posterior_predictive(trace)
        SiteTrace[s]=trace
        if (idx+1)%10==0 or idx==len(Site)-1:
            pklname="dict"+str(number)+".pkl"
            with open(pklname, 'wb') as file:
                pickle.dump(SiteTrace, file)
            SiteTrace={}
            number+=1
#         SiteTrace[s]={'alpha':trace['alpha'].mean(),'beta0':trace['beta'][0].mean(),'beta1':trace['beta'][1].mean()}

/opt/conda/lib/python3.10/site-packages/deprecat/classic.py:215: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  return wrapped_(*args_, **kwargs_)


/opt/conda/lib/python3.10/site-packages/deprecat/classic.py:215: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  return wrapped_(*args_, **kwargs_)


/opt/conda/lib/python3.10/site-packages/deprecat/classic.py:215: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  return wrapped_(*args_, **kwargs_)


/opt/conda/lib/python3.10/site-packages/deprecat/classic.py:215: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  return wrapped_(*args_, **kwargs_)


/opt/conda/lib/python3.10/site-packages/deprecat/classic.py:215: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  return wrapped_(*args_, **kwargs_)


...

/opt/conda/lib/python3.10/site-packages/deprecat/classic.py:215: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  return wrapped_(*args_, **kwargs_)


/opt/conda/lib/python3.10/site-packages/deprecat/classic.py:215: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  return wrapped_(*args_, **kwargs_)


/opt/conda/lib/python3.10/site-packages/deprecat/classic.py:215: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  return wrapped_(*args_, **kwargs_)


/opt/conda/lib/python3.10/site-packages/deprecat/classic.py:215: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  return wrapped_(*args_, **kwargs_)


/opt/conda/lib/python3.10/site-packages/deprecat/classic.py:215: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  return wrapped_(*args_, **kwargs_)


In [15]:
ref={}
for i in range(1,9):
    filename="dict"+str(i)+".pkl"
    with open(filename,"rb") as file:
        ref.update(pickle.load(file))

In [16]:
def calculate_result(row):
    site=int(row['SiteId'])
    a=ref[site]['alpha'].mean()
    b0=ref[site]['beta'][0].mean()
    b1=ref[site]['beta'][1].mean()
    b2=ref[site]['beta'][2].mean()
    result = a + b0 * row[features[0]] + b1 * row[features[1]]+  b2 * row[features[2]]
    return result
df['result'] = df.apply(calculate_result, axis=1)

In [17]:
mse = mean_squared_error(df['PM2.5'].values, df['result'].values)
print("training mse:",mse)

training mse: 436.1765150100062


In [18]:
truth=preprocessing(truth)
truth['result'] = truth.apply(calculate_result, axis=1)
mse = mean_squared_error(truth['PM2.5'].values, truth['result'].values)
print("total mse:",mse)

<ipython-input-14-7e5edb5d69f7>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.dropna(inplace=True)


total mse: 515.4380664888341
